## Get Data

In case you have not run the `1 - get_data.ipynb`, run the following cell.

In [1]:
!wget http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv
import get_data

get_data.get_quora_data()

--2022-01-16 15:58:20--  http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv
Resolving qim.fs.quoracdn.net (qim.fs.quoracdn.net)... 151.101.53.2
Connecting to qim.fs.quoracdn.net (qim.fs.quoracdn.net)|151.101.53.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58176133 (55M) [text/tab-separated-values]
Saving to: ‘quora_duplicate_questions.tsv.1’

quora_duplicate_que 100%[===================>]  55.48M  30.8MB/s    in 1.8s    

2022-01-16 15:58:23 (30.8 MB/s) - ‘quora_duplicate_questions.tsv.1’ saved [58176133/58176133]

train.txt has been saved!
dev.txt has been saved!
test.txt has been saved!


## Preprocess and numericalize text data

In case you have not run the `2 - preprocess_data.ipynb`, run the following cell.

In [2]:
from utils import *

# ---- load dataset ----
train_set, dev_set, test_set = load_dataset(['train.txt', 'dev.txt', 'test.txt'])

# ---- numericalize the train set ----
V = TextVectorizer(tokenize) 
text = gather_text(train_set) # for collecting texts from train set
V.build_vocab(text) # for building mapping vocab_to_idx dictionary and text_encoder
train_set_encoded = list(encode_dataset(train_set, encoder=V.text_encoder)) # encodoing train set
dev_set_encoded = list(encode_dataset(dev_set, encoder=V.text_encoder)) # encodoing dev set for validation
test_set_encoded  = list(encode_dataset(test_set, encoder=V.text_encoder)) # encodoing dev set for prediction

# ---- build mini batches for the train and dev set ----
train_set_batched = build_batches(train_set_encoded, batch_size=64, include_seq_len=False)
dev_set_batched = build_batches(dev_set_encoded, batch_size=64, include_seq_len=False)
test_set_batched = build_batches(test_set_encoded, batch_size=64, include_seq_len=False)

Two vocabulary dictionaries have been built!
Please call X.vocab_to_idx | X.idx_to_vocab to find out more where [X] stands for the name you used for this TextVectorizer class.


## Training and evaluating models 

### BoW (Bag of Words) model

#### Training

In [3]:
from paddle_models.BoW import BoW
import paddle

In [4]:
def get_model(model):
    model = paddle.Model(model)
    optimizer = paddle.optimizer.Adam(
    parameters=model.parameters(), learning_rate=5e-4)
    criterion = paddle.nn.CrossEntropyLoss()
    metric = paddle.metric.Accuracy()
    model.prepare(optimizer, criterion, metric)
    return model

In [5]:
model = BoW(len(V.vocab_to_idx), 2)
model = get_model(model)
%time model.fit(train_set_batched, dev_set_batched, epochs=5, save_dir='./ckpt', verbose=1)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/5
step 30/47 [==================>...........] - loss: 0.6676 - acc: 0.5865 - ETA: 0s - 6ms/step

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  return (isinstance(seq, collections.Sequence) and


step 47/47 [==============================] - loss: 0.6391 - acc: 0.5983 - 6ms/step          
save checkpoint at /Users/wzx/Documents/GitHub/text_matching/ckpt/0
Eval begin...
step 16/16 [==============================] - loss: 0.5784 - acc: 0.6450 - 2ms/step          
Eval samples: 1000
Epoch 2/5
step 47/47 [==============================] - loss: 0.4903 - acc: 0.8040 - 6ms/step          
save checkpoint at /Users/wzx/Documents/GitHub/text_matching/ckpt/1
Eval begin...
step 16/16 [==============================] - loss: 0.4543 - acc: 0.6630 - 2ms/step          
Eval samples: 1000
Epoch 3/5
step 47/47 [==============================] - loss: 0.3227 - acc: 0.9003 - 6ms/step          
save checkpoint at /Users/wzx/Documents/GitHub/text_matching/ckpt/2
Eval begin...
step 16/16 [==============================] - loss: 0.4664 - acc: 0.6420 - 2ms/step          
Eval samples: 1000
Epoch 4/5
step 47/47 [==============================] - loss: 0.1736 - acc: 0.9633 - 6ms/step          
save chec

#### Evaluation on the test set

In [6]:
model.evaluate(test_set_batched)

Eval begin...
step 10/16 - loss: 0.9671 - acc: 0.6203 - 2ms/step
step 16/16 - loss: 1.0170 - acc: 0.6120 - 2ms/step
Eval samples: 1000


{'loss': [1.0170405], 'acc': 0.612}

### CNN (Convolutional Neural Network) model

#### Training

In [7]:
from paddle_models.CNN import CNN

model = CNN(len(V.vocab_to_idx), 2)
model = get_model(model)
%time model.fit(train_set_batched, dev_set_batched, epochs=5, save_dir='./ckpt', verbose=1)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/5
step 47/47 [==============================] - loss: 0.6877 - acc: 0.5500 - 71ms/step          
save checkpoint at /Users/wzx/Documents/GitHub/text_matching/ckpt/0
Eval begin...
step 16/16 [==============================] - loss: 0.6667 - acc: 0.5670 - 26ms/step          
Eval samples: 1000
Epoch 2/5
step 47/47 [==============================] - loss: 0.5575 - acc: 0.7697 - 70ms/step          
save checkpoint at /Users/wzx/Documents/GitHub/text_matching/ckpt/1
Eval begin...
step 16/16 [==============================] - loss: 0.5453 - acc: 0.6480 - 27ms/step          
Eval samples: 1000
Epoch 3/5
step 47/47 [==============================] - loss: 0.3051 - acc: 0.8747 - 72ms/step          
save checkpoint at /Users/wzx/Documents/GitHub/text_matching/ckpt/2
Eval begin...
step 16/16 [==============================] - loss: 0.4356 - acc: 0.6790 - 26ms/step          
Eval s

#### Evaluation on the test set

In [8]:
model.evaluate(test_set_batched)

Eval begin...
step 10/16 - loss: 0.9078 - acc: 0.6422 - 26ms/step
step 16/16 - loss: 1.0210 - acc: 0.6330 - 26ms/step
Eval samples: 1000


{'loss': [1.0210375], 'acc': 0.633}

## RNN (Recurrent neural network) models

As the RNN models also take as an input the sequence length, we need to re-encode the train set, dev set, and test set. 

In [9]:
# ---- build mini batches for the train and dev set ----
train_set_batched = build_batches(train_set_encoded, batch_size=64, include_seq_len=True)
dev_set_batched = build_batches(dev_set_encoded, batch_size=64, include_seq_len=True)
test_set_batched = build_batches(test_set_encoded, batch_size=64, include_seq_len=True)

### Simple RNN model

#### Training

In [10]:
from paddle_models.S_RNN import SimpleRNN

model = SimpleRNN(len(V.vocab_to_idx), 2)
model = get_model(model)
%time model.fit(train_set_batched, dev_set_batched, epochs=5, save_dir='./ckpt', verbose=1)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/5
step 47/47 [==============================] - loss: 0.6916 - acc: 0.5267 - 20ms/step          
save checkpoint at /Users/wzx/Documents/GitHub/text_matching/ckpt/0
Eval begin...
step 16/16 [==============================] - loss: 0.6882 - acc: 0.5310 - 8ms/step          
Eval samples: 1000
Epoch 2/5
step 47/47 [==============================] - loss: 0.5643 - acc: 0.6923 - 20ms/step          
save checkpoint at /Users/wzx/Documents/GitHub/text_matching/ckpt/1
Eval begin...
step 16/16 [==============================] - loss: 0.6363 - acc: 0.5700 - 7ms/step          
Eval samples: 1000
Epoch 3/5
step 47/47 [==============================] - loss: 0.1211 - acc: 0.8907 - 22ms/step          
save checkpoint at /Users/wzx/Documents/GitHub/text_matching/ckpt/2
Eval begin...
step 16/16 [==============================] - loss: 0.8905 - acc: 0.5650 - 8ms/step          
Eval samp

#### Evaluation on the test set

In [11]:
model.evaluate(test_set_batched)

Eval begin...
step 10/16 - loss: 1.3285 - acc: 0.5578 - 7ms/step
step 16/16 - loss: 1.3155 - acc: 0.5600 - 8ms/step
Eval samples: 1000


{'loss': [1.3154628], 'acc': 0.56}

### GRU (Gated recurrent units) model 

#### Training

In [12]:
from paddle_models.GRU import GRU

model = GRU(len(V.vocab_to_idx), 2)
model = get_model(model)
%time model.fit(train_set_batched, dev_set_batched, epochs=5, save_dir='./ckpt', verbose=1)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/5
step 47/47 [==============================] - loss: 0.6896 - acc: 0.5137 - 44ms/step          
save checkpoint at /Users/wzx/Documents/GitHub/text_matching/ckpt/0
Eval begin...
step 16/16 [==============================] - loss: 0.6917 - acc: 0.5840 - 17ms/step          
Eval samples: 1000
Epoch 2/5
step 47/47 [==============================] - loss: 0.5612 - acc: 0.6713 - 41ms/step          
save checkpoint at /Users/wzx/Documents/GitHub/text_matching/ckpt/1
Eval begin...
step 16/16 [==============================] - loss: 0.6567 - acc: 0.6270 - 18ms/step          
Eval samples: 1000
Epoch 3/5
step 47/47 [==============================] - loss: 0.2581 - acc: 0.8450 - 47ms/step          
save checkpoint at /Users/wzx/Documents/GitHub/text_matching/ckpt/2
Eval begin...
step 16/16 [==============================] - loss: 0.6945 - acc: 0.6150 - 17ms/step          
Eval s

#### Evaluation on the test set

In [13]:
model.evaluate(test_set_batched)

Eval begin...
step 10/16 - loss: 1.4538 - acc: 0.6094 - 18ms/step
step 16/16 - loss: 1.0900 - acc: 0.6120 - 18ms/step
Eval samples: 1000


{'loss': [1.0899506], 'acc': 0.612}

### LSTM (Long short-term memory) model

#### Training

In [14]:
from paddle_models.LSTM import LSTM

model = LSTM(len(V.vocab_to_idx), 2)
model = get_model(model)
%time model.fit(train_set_batched, dev_set_batched, epochs=5, save_dir='./ckpt', verbose=1)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/5
step 47/47 [==============================] - loss: 0.6913 - acc: 0.4953 - 55ms/step          
save checkpoint at /Users/wzx/Documents/GitHub/text_matching/ckpt/0
Eval begin...
step 16/16 [==============================] - loss: 0.6941 - acc: 0.5000 - 21ms/step          
Eval samples: 1000
Epoch 2/5
step 47/47 [==============================] - loss: 0.6297 - acc: 0.6033 - 52ms/step          
save checkpoint at /Users/wzx/Documents/GitHub/text_matching/ckpt/1
Eval begin...
step 16/16 [==============================] - loss: 0.6599 - acc: 0.6310 - 20ms/step          
Eval samples: 1000
Epoch 3/5
step 47/47 [==============================] - loss: 0.3528 - acc: 0.8113 - 54ms/step          
save checkpoint at /Users/wzx/Documents/GitHub/text_matching/ckpt/2
Eval begin...
step 16/16 [==============================] - loss: 0.6847 - acc: 0.6540 - 20ms/step          
Eval s

#### Evaluation on the test set

In [15]:
model.evaluate(test_set_batched)

Eval begin...
step 10/16 - loss: 1.3624 - acc: 0.6078 - 22ms/step
step 16/16 - loss: 1.3373 - acc: 0.6220 - 21ms/step
Eval samples: 1000


{'loss': [1.3372626], 'acc': 0.622}